# 설치 및 기본환경
- conda activate webcrawl_env 활성화
- pip install Pillow
- 기타 필요한 라이브러리 import

In [1]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os

# requests.get
- requests로 지정한 url을 get. html를 가져오고 이를 moving_ranking에 변수로 저장
- image텍스트를 image_dir 변수에 저장한다.

# 디렉토리 만들기
- 파이썬 os모듈에서 디렉토리 만들기, image_dir폴더가 이미 있으면 통과, 없으면 생성하라.
- 파일 이름에 쓸 수 없는 문자를 미리 규정, \ / : " * ? < > |은 윈도우에서는 파일이름으로 사용할 수 없다. 
- r'는 raw string의 약자, 특수문자로 해석하지 말고 그대로 써라. 파이썬에서 \n은 줄바꿈 표시, 즉 앞으로 파일이름에 쓸 문자에서 \가 나오면 줄바꿈 의미해석이 아니라 문자 그대로 써라는 뜻

In [4]:
movie_ranking = requests.get("https://www.moviechart.co.kr/rank/realtime/index/image")
image_dir = 'images'
os.makedirs(image_dir, exist_ok=True)
pattern = r'[\\/:"*?<>|]'


# 영화정보 출력
- 변수에 저장한 url에 상태를 물어보고 상태값을 받아오는데 그때 200(성공)하면 해당 텍스트를 출력하라. 

## url 상태값이 성공이면 정보 출력 조건
- 따라서 if문 안으로 들어감
## BeautifulSoup사용 파서 처리
- 변수로 지정된 movie_ranking은 단순 객체, 그안에 있는 html 원본 데이터 꺼내기 위해 content 사용, .text는 문자열(str) 형태 (디코딩된 텍스트)이다. 실제 content 많이 사용. BeautifulSoup이 바이트 데이터를 잘 처리하기 때문
## 타이틀과 이미지 찾기
- css 선택자로 html요소를 찾는다. 
- movieBox-list 클래스 밑에 movie-title 클래스 밑에  있는 a태그를 찾아라
## 두개 리스트를 같이 묶어서 보여주기
- for문에서 zip은 두개의 리스트를 같은 순서끼리 묶어준다. 제목과 이미르 하나를 동시에 꺼내라
- 이를 반복하여 출력하라
- html안에 있는 문서 경로에서 src뒤에 붙은 이미지 경로를 가져와서 출력하라.
## 파이썬 메모리에 저장
- url뒤에 이미지 경로 붙여서 이미지 데이터를 가져와 파이썬 메모리에 해당 변수로 저장해라
## 이미지 바이트덩어리를 이미지 객체로 변환
- 이미지 파일의 바이너리 데이터를 파일처럼 다루게 해주는 BytesI0 실행
- 즉, 메모리 안의 바이트 덩어리를 가짜 파일처럼 만들어주는 BytesI0 실행
- 만들어진 파일을 열어서 이미지 객체로 만들어주는 Image.open 사용, 이것을 img에 넣어라
## 파일에 특수문자 제거 처리
- 영화 제목에 pattern에 지정된 특수문자를 공백으로 바꿔서 image_filename으로 저장
## 경로, 파일이름 연결처리
- os.path.join으로 이미지폴더, 특수문자 제거된 이미지파일.png 이 이미지를 저장. 컴터에 저장됨. 

In [8]:
if movie_ranking.status_code == 200:
  print("영화 정보를 출력합니다.")
  soup = BeautifulSoup(movie_ranking.content, 'html.parser')
  movie_title_list = soup.select(".movieBox-list .movie-title a") # 영화 이름 <a> 요소 목록
  movie_image_list = soup.select(".movieBox-list .movieBox-item img") #img 태그 요소
  print(f"수집한 영화 수: {len(movie_title_list)}")
  for movie_title, movie_image in zip(movie_title_list, movie_image_list):
    print(movie_title.text, movie_image.get('src')) 
    image_src = movie_image.get('src')
    image_response = requests.get("https://www.moviechart.co.kr" + image_src)
    img = Image.open(BytesIO(image_response.content))
    image_filename = re.sub(pattern, '', movie_title.text) #파일이름 지정, 정규식
    img.save(os.path.join(image_dir, image_filename + ".png"))
    print(movie_title.text, )  
else:
  print("페이지에 연결할 수 없습니다.")

영화 정보를 출력합니다.
수집한 영화 수: 20
왕과 사는 남자 /thumb?width=178&height=267&m_code=20242837&source=https://admin.moviechart.co.kr/assets/upload/movie/260108003526_8322.jpg
왕과 사는 남자
휴민트 /thumb?width=178&height=267&m_code=20241266&source=https://admin.moviechart.co.kr/assets/upload/movie/260112024651_6301.jpg
휴민트
초속 5센티미터 /thumb?width=178&height=267&m_code=20259583&source=https://admin.moviechart.co.kr/assets/upload/movie/260213052538_7190.jpg
초속 5센티미터
너자 2 /thumb?width=178&height=267&m_code=20261181&source=https://admin.moviechart.co.kr/assets/upload/movie/260202063756_4036.jpg
너자 2
슬라이드 스트럼 뮤트 /thumb?width=178&height=267&m_code=20261450&source=https://admin.moviechart.co.kr/assets/upload/movie/260219042530_2928.jpg
슬라이드 스트럼 뮤트
넘버원 /thumb?width=178&height=267&m_code=20252373&source=https://admin.moviechart.co.kr/assets/upload/movie/260123065552_5690.jpg
넘버원
햄넷 /thumb?width=178&height=267&m_code=20250365&source=https://admin.moviechart.co.kr/assets/upload/movie/260202063420_3704.jpg
햄넷
부흥 /thumb?wid